In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
PATH = "/content/drive/MyDrive/Fairness/data/MBIC.xlsx"
df = pd.read_excel(PATH)

columns_to_remove = ['Unnamed: 0', 'news_link', 'outlet', 'topic', 'type',
                     'group_id', 'num_sent', 'Label_opinion', 'article',
                     'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13']


df = df.drop(columns=columns_to_remove, errors='ignore')  
df = df[df['Label_bias'] == 'Biased']
df = df.iloc[:100]

In [ ]:
pip install --upgrade openai

In [ ]:
import pandas as pd
import openai


In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ''  # Replace  with your actual OpenAI API key.

In [ ]:
df_bias = pd.read_excel('/content/drive/MyDrive/Fairness/data/bias_word_lexicon.xlsx')

In [ ]:
biased_words = df_bias['slavishness'].tolist()

# Zero-shot

In [ ]:
from openai import OpenAI


def debias_text_conversation(text, biased_words):

    
    conversation = [
        {"role": "system", "content": "As an AI, you are aware of various biases that can appear in language. These biases can be related to gender, race, age, or cultural background. It is important to address these biases to ensure neutrality and inclusiveness. Some words that may reflect bias include: " + ", ".join(biased_words) + "." + "You are an assistant trained to identify and remove biases from text. Make sure the text is neutral, inclusive, and respects all individuals regardless of their gender, race, age, or cultural background."},
        {"role": "user", "content": f"Please debias the following text: {text}"}
    ]

    client = OpenAI()


    
    response = client.chat.completions.create(
        model="gpt-4", 
        messages=conversation
    )

    
    if response.choices and response.choices[0].message:
        
        debiased_text = response.choices[0].message.content
    else:
        debiased_text = "Debiasing failed"  

    return debiased_text



df['debias_text'] = df['sentence'].apply(lambda x: debias_text_conversation(x, biased_words))


df.to_csv('/content/drive/MyDrive/Fairness/data/Knowledge-zeroShot-Debiasing.csv', index=False)


# 2-shots

In [ ]:
from openai import OpenAI


def debias_text_conversation(text, biased_words):

   
    conversation = [
        {"role": "system", "content": "As an AI, you are aware of various biases that can appear in language. These biases can be related to gender, race, age, or cultural background. It is important to address these biases to ensure neutrality and inclusiveness. Some words that may reflect bias include: " + ", ".join(biased_words) + "." + "You are an assistant trained to identify and remove biases from text. Make sure the text is neutral, inclusive, and respects all individuals regardless of their gender, race, age, or cultural background. For example, if the text says 'Billie Eilish issues apology for mouthing an anti-Asian derogatory term in a resurfaced video', you should debias it to Billie Eilish issues apology for using an anti-Asian remark in a resurfaced video'. If the text says 'If he is elected, he would be the first person of color in the White House.', debias it to 'lf he is elected, he would be the first African-American in the White House.'."},
        {"role": "user", "content": f"Please debias the following text: {text}"}
    ]

    client = OpenAI()


  
    response = client.chat.completions.create(
        model="gpt-4",  
        messages=conversation
    )

    
    if response.choices and response.choices[0].message:
        
        debiased_text = response.choices[0].message.content
    else:
        debiased_text = "Debiasing failed"  

    return debiased_text



df['debias_text'] = df['sentence'].apply(lambda x: debias_text_conversation(x, biased_words))


df.to_csv('/content/drive/MyDrive/Fairness/data/Knowledge-TwoShots-Debiasing.csv', index=False)

# 4-shots

In [ ]:
from openai import OpenAI


def debias_text_conversation(text, biased_words):

    
    conversation = [
        {"role": "system", "content": "As an AI, you are aware of various biases that can appear in language. These biases can be related to gender, race, age, or cultural background. It is important to address these biases to ensure neutrality and inclusiveness. Some words that may reflect bias include: " + ", ".join(biased_words) + "." + "You are an assistant trained to identify and remove biases from text. Make sure the text is neutral, inclusive, and respects all individuals regardless of their gender, race, age, or cultural background. For example, if the text says 'Billie Eilish issues apology for mouthing an anti-Asian derogatory term in a resurfaced video', you should debias it to Billie Eilish issues apology for using an anti-Asian remark in a resurfaced video'. If the text says 'If he is elected, he would be the first person of color in the White House.', debias it to 'lf he is elected, he would be the first African-American in the White House.’, if the text says ‘He has had the physical handicap since he was 5 years old.', you should debias it to ‘He has had the physical impairment since he was 5 years old.'. If the text says’ There are many elderly people in our town.', debias it to’ There are many senior citizens (or seniors) in our town.'."},
        {"role": "user", "content": f"Please debias the following text: {text}"}
    ]

    client = OpenAI()


   
    response = client.chat.completions.create(
        model="gpt-4",  
        messages=conversation
    )

   
    if response.choices and response.choices[0].message:
        
        debiased_text = response.choices[0].message.content
    else:
        debiased_text = "Debiasing failed"  

    return debiased_text



df['debias_text'] = df['sentence'].apply(lambda x: debias_text_conversation(x, biased_words))


df.to_csv('/content/drive/MyDrive/Fairness/data/Knowledge-FourShots-Debiasing.csv', index=False)